In [1]:
import h5py
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import sys
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
import helper.utils as UTIL

/home/jackalhan/Development/github/more_meaningful_representations/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## DATASET ARE LOADED WITH TOKENIZED FORM
### THIS DOES NOT NEED TO BE CHANGED

In [21]:
DIR = '/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/'
args={
'train_dataset_file': 'train-v1.1.json',
'dev_dataset_file': 'dev-v1.1.json',
}
dev_tokenized_questions, \
dev_tokenized_paragraphs,\
dev_questions_nontokenized,\
dev_paragraphs_nontokenized = UTIL.prepare_squad_objects(os.path.join(DIR, args['dev_dataset_file']), 'DEV')

print('Len of Dev Questions: {}, Len of Dev Paragraphs: {}'.format(len(dev_tokenized_questions),
                                                                  len(dev_tokenized_paragraphs)))

train_tokenized_questions, \
train_tokenized_paragraphs,\
train_questions_nontokenized,\
train_paragraphs_nontokenized = UTIL.prepare_squad_objects(os.path.join(DIR, args['train_dataset_file']), 'TRAIN')
print('Len of Train Questions: {}, Len of Train Paragraphs: {}'.format(len(train_tokenized_questions),
                                                                  len(train_tokenized_paragraphs)))



****************************************************************************************************
Parsing Started
Generating DEV examples...


100%|██████████| 48/48 [00:07<00:00,  6.08it/s]


10570 questions in total
# of Paragraphs in DEV : 2067
# of Questions in DEV : 10570
# of Q_to_P DEV : 10570
--------------------
Paragraphs: Tokenization and Saving Tokenization Started in DEV
# of Tokenized Paragraphs in DEV : 2067
--------------------
Questions: Tokenization and Saving Tokenization Started in DEV
# of Tokenized Questions in DEV : 10570
Parsing Ended in 0.21666666666666667 minutes
****************************************************************************************************


100%|██████████| 442/442 [01:11<00:00,  6.16it/s]


Len of Dev Questions: 10570, Len of Dev Paragraphs: 2067
****************************************************************************************************
Parsing Started
Generating TRAIN examples...
87599 questions in total
# of Paragraphs in TRAIN : 18896
# of Questions in TRAIN : 87599
# of Q_to_P TRAIN : 87599
--------------------
Paragraphs: Tokenization and Saving Tokenization Started in TRAIN
# of Tokenized Paragraphs in TRAIN : 18896
--------------------
Questions: Tokenization and Saving Tokenization Started in TRAIN
# of Tokenized Questions in TRAIN : 87599
Parsing Ended in 2.1333333333333333 minutes
****************************************************************************************************
Len of Train Questions: 87599, Len of Train Paragraphs: 18896


In [23]:
tokenized_questions = dev_tokenized_questions + train_tokenized_questions
tokenized_paragraphs = dev_tokenized_paragraphs + train_tokenized_paragraphs
questions_nontokenized = dev_questions_nontokenized + train_questions_nontokenized
paragraphs_nontokenized = dev_paragraphs_nontokenized + train_paragraphs_nontokenized

#### YOU CAN EITHER USE ONE OF THE DEV/TRAIN OR COMBINATION  TOKENS ABOVE FOR THE PURPOSE

### THE FOLLOWING PART, NEEDS TO BE RUN FOR EACH DATA FOLDER IN PARAM such as ULTIMATE_OLD_API_no_IDF_with_1_0_0

In [24]:
PARAMS_PATH= "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/params.json"
params = UTIL.Params(PARAMS_PATH)
_base_path = os.path.join(DIR, params.executor['data_dir'])
print(_base_path)

/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/data/DEV_OLD_API_no_IDF_with_1_0_0


In [25]:
KN_FILE_NAMES =UTIL.get_file_key_names_for_execution(params)
print('Execute for {}'.format(KN_FILE_NAMES['DIR']))
data_path = os.path.join(_base_path, KN_FILE_NAMES['DIR'])
print(data_path)

Execute for question_training
/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/data/DEV_OLD_API_no_IDF_with_1_0_0/question_training


### 1) TRAINING FILES

In [30]:
train_target_embeddings=UTIL.load_embeddings(os.path.join(data_path,
                                                   params.files["train_loss"][KN_FILE_NAMES["KN_TARGET_EMBEDDINGS"]]))

train_source_indx =UTIL.load_embeddings(os.path.join(data_path,
                                                   params.files["train_loss"][KN_FILE_NAMES["KN_SOURCE_IDX"]])).astype(int)
train_source_labels =UTIL.load_embeddings(os.path.join(data_path,
                                                   params.files["train_loss"][KN_FILE_NAMES["KN_SOURCE_LABELS"]])).astype(int)

In [27]:
train_source_texts = []
for indx in tqdm(train_source_indx):
    train_question = questions_nontokenized[indx]
    train_questions.append(train_question)

100%|██████████| 6725/6725 [00:08<00:00, 789.58it/s] 


In [29]:
train_target_embeddings

array([[-0.08684976, -0.04605952, -0.09545521, ...,  0.01791359,
         0.0098526 ,  0.13276504],
       [-0.09024484, -0.05345621,  0.04380247, ...,  0.09215055,
         0.10844743,  0.16277342],
       [-0.10331678,  0.06271477, -0.03893897, ...,  0.05493199,
         0.01267613,  0.06338305],
       ...,
       [-0.10705659, -0.02247855, -0.12750474, ...,  0.06168772,
        -0.08351889,  0.01065556],
       [-0.04641121,  0.03306587, -0.1092279 , ..., -0.03879117,
        -0.05317794, -0.09994104],
       [-0.04497295, -0.02209207, -0.21226965, ..., -0.0476587 ,
         0.13204622, -0.00755106]], dtype=float32)

In [1]:
def get_audio(filename):
    sr, audio = read(filename)
    audio = audio.astype(float)
    audio = audio - audio.min()
    audio = audio / (audio.max() - audio.min())
    audio = (audio - 0.5) * 2
    return sr, audio

In [3]:
from scipy.io.wavfile import read, write

In [12]:
sr, audio = get_audio("/home/jackalhan/Downloads/a2002011001-e02.wav")

In [19]:
import numpy as np

In [21]:
n_epochs = 20
frame_size = 2048
frame_shift = 512

In [34]:
training_audio =np.random.randn(10584000,2)

In [35]:
training_audio.shape

(10584000, 2)

In [36]:
training_audio = training_audio[:sr*240]

In [37]:
training_audio.shape

(10584000, 2)

In [38]:
2e-4

0.0002

In [1]:
import tensorflow as tf
import numpy as np
import h5py
import sys
import os
path = !echo ${VIRTUAL_ENV}
path = os.path.join(path[0], '..')
sys.path.append(path)
import helper.utils as UTIL

/home/jackalhan/Development/github/more_meaningful_representations/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [52]:
class generator:
    def __init__(self, file, table_name='embeddings'):
        self.file = file
        self.table_name = table_name
    def __call__(self):
        with h5py.File(self.file, 'r') as hf:
            for im in hf[self.table_name]:
                yield im

def _read_dataset(file_path, embedding_dim, table_name='embeddings', data_type=tf.float32):
    if embedding_dim is not None:
        ds = tf.data.Dataset.from_generator(
            generator(file_path, table_name),
            data_type,
            tf.TensorShape([embedding_dim, ]))
    else:
        ds = tf.data.Dataset.from_generator(
            generator(file_path, table_name),
            data_type,
            tf.TensorShape([]))
    return ds

def _parser_non_conv(source_embeddings,
                     target_embeddings,
                     target_labels):
    features = {"source_embeddings": source_embeddings}
    labels = {"target_embeddings": target_embeddings,
              "target_labels": target_labels}
    return features, labels

def _repeater_fn(target_labels, repeat_list):
    if target_labels not in repeat_list:
        return 1
    else:
        return 0

In [26]:
path = "/home/jackalhan/Development/github/more_meaningful_representations/squad/train/improvement/data/ULTIMATE_OLD_API_with_IDF_with_1_0_0/"
_train_source_embeddings = os.path.join(path, "question_training","train_question_embeddings.hdf5")
_train_target_embeddings = os.path.join(path,"question_training", "train_paragraph_embeddings.hdf5")
_train_source_labels = os.path.join(path,"question_training", "train_question_labels.hdf5")
_train_source_embeddings = _read_dataset(_train_source_embeddings, 1024)
_train_target_embeddings = _read_dataset(_train_target_embeddings, 1024)
_train_source_labels = _read_dataset(_train_source_labels, None, data_type=tf.int64)

In [209]:
def input_fn(class_size=20963, batch=512, buffer_size=84787):
    dataset = tf.data.Dataset.zip((_train_source_embeddings, _train_target_embeddings, _train_source_labels))
    target_list_np = np.random.choice(class_size, batch, replace=False)    
    dataset = dataset.map(_parser_non_conv)      
    #dataset = dataset.shuffle(buffer_size=buffer_size) 
    for i, t in enumerate(target_list_np):        
        if i == 0:
            dataset_ = dataset.filter(lambda features, labels: tf.equal(labels['target_labels'], t)).take(1)
        else:
            dataset_ = dataset_.concatenate(dataset.filter(lambda features, labels: tf.equal(labels['target_labels'], t)).take(1))
    dataset = dataset_.shuffle(batch)
    dataset = dataset_.batch(batch)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()
    return next_element, target_list_np

In [210]:
_next, par = input_fn(batch=4)
with tf.Session() as sess:
    value = sess.run(_next)

In [211]:
value

({'source_embeddings': array([[-1.1460828 ,  1.4028684 , -0.8121466 , ..., -0.75059336,
          -1.9347275 ,  1.570022  ],
         [-2.88856   ,  0.7939497 , -0.5279569 , ..., -1.0717467 ,
           0.7037743 ,  2.2476268 ],
         [-1.5454668 , -0.9402997 , -1.7358644 , ..., -0.07986278,
           2.4861035 , -2.1018429 ],
         [-0.02267063,  2.6228843 , -2.280973  , ...,  1.4978906 ,
           1.65165   ,  0.39760858]], dtype=float32)},
 {'target_embeddings': array([[ 0.21672936,  0.60549283, -0.5099839 , ...,  1.2391334 ,
           0.22140406, -0.2875904 ],
         [-0.7677321 , -0.18804908,  0.5015572 , ...,  0.33911023,
           1.1989979 , -0.92873794],
         [-0.07599598,  0.3036891 , -0.23900305, ...,  0.8791997 ,
           0.37414524, -0.3102086 ],
         [-1.295609  ,  0.17239891,  1.1195781 , ...,  1.0096842 ,
           1.1128237 , -0.6416027 ]], dtype=float32),
  'target_labels': array([ 1292, 18653,  6116,  9956])})

In [212]:
par

array([ 1292, 18653,  6116,  9956])

In [180]:
labels = np.random.randint(25, size=100000)
data = np.random.randint(100000, size=100000)
uniqs = tf.constant(np.random.randint(50, size=50))
def input_fn():    
    batch_size = 25
    ds = tf.data.Dataset.from_tensor_slices((data, labels))#.repeat(lambda data, labels: tf.constant(1))
    ds = ds.shuffle(100000)
    for indx, l in enumerate(np.unique(labels)):
        #print("{} -> {}".format(indx, l))
        if indx == 0:
            ds_a = ds.filter(lambda data, label: tf.equal(label, l)).take(1)

        else:
            ds_a = ds_a.concatenate(ds.filter(lambda data, label: tf.equal(label, l)).take(1))
    # def mapping(data, labels):
    #     output, idx = tf.unique_with_counts(labels, out_idx=tf.int64)
    # output_labels = ds.map(mapping)
    #ds_a = ds_a.shuffle(2)
    ds_a = ds_a.batch(batch_size)
    ds = ds.prefetch(1)
    ds_x = ds_a.make_one_shot_iterator()
    next_element = ds_x.get_next()
    return next_element


In [167]:
sess = tf.InteractiveSession()

/home/jackalhan/Development/github/more_meaningful_representations/venv/lib/python3.5/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [181]:
next_element = input_fn()
sess.run(next_element)

KeyboardInterrupt: 

In [142]:
labels

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,